In [2]:
%matplotlib inline

import folium
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import nltk
import numpy as np
import operator 
import os
import pandas as pd
import pytz
import random
import re
import seaborn as sns
import string
import sys
import time
import vincent

from collections import Counter
from collections import defaultdict
from datetime import datetime
from matplotlib import dates
from matplotlib import rcParams
from matplotlib.ticker import MaxNLocator
from mpltools import style
from nltk import FreqDist
from nltk import bigrams 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from os import path
from pandas.tseries.resample import TimeGrouper
from pandas.tseries.offsets import DateOffset
from scipy.misc import imread
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

nltk.download('punkt')
nltk.download('mac_morpho')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/thiago/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package mac_morpho to
[nltk_data]     /Users/thiago/nltk_data...
[nltk_data]   Package mac_morpho is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/thiago/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
sns.set_palette("deep", desat=.6)
sns.set_context(rc={"figure.figsize": (8, 4)})
style.use('ggplot')
rcParams['axes.labelsize'] = 9
rcParams['xtick.labelsize'] = 9
rcParams['ytick.labelsize'] = 9
rcParams['legend.fontsize'] = 7
# rcParams['font.serif'] = ['Computer Modern Roman']
rcParams['font.serif'] = ['Ubuntu']
rcParams['text.usetex'] = False
rcParams['figure.figsize'] = 20, 10

# pd.set_option('display.max_colwidth', 200)
# pd.options.display.mpl_style = 'default'
# matplotlib.style.use('ggplot')
# sns.set_context('talk')
# sns.set_style('whitegrid')

print 'OK!'

OK!


In [4]:
with open('data-docker/peda.json') as data_file:    
    data = json.load(data_file)


print 'OK!'

OK!


### Algumas funções

In [87]:
def datetimeify(df):
    df['created_at'] = pd.DatetimeIndex(df.created_at)
    return df

In [88]:
def sentiment(df):
    text = df.dropna(subset=['text']).text
    sentiment = text.apply(lambda text: TextBlob(text).sentiment)
    df['polarity'] = sentiment.apply(lambda sentiment: sentiment.polarity)
    df['subjectivity'] = sentiment.apply(lambda sentiment: sentiment.subjectivity)
    return df

In [89]:
def influence(df):
    internal = np.sqrt(df.user_followers_count.apply(lambda x: x + 1))
    external = np.sqrt(df.retweet_count.apply(lambda x: x + 1))
    df['influence'] = internal * external
    return df

In [90]:
def influenced_polarity(df):
    df['influenced_polarity'] = df.polarity * df['influence']
    return df

In [91]:
def georeference(df):
    def place_to_coordinate(place_str, kind):
        if pd.isnull(place_str):
            return float('nan')
        number_matcher = r'(-?\d+\.\d+)[,\]]'
        coordinates = re.findall(number_matcher, place_str)
        coordinate = tuple(float(n) for n in coordinates[:2])

        if kind == 'longitude':
            return coordinate[0]
        elif kind == 'latitude':
            return coordinate[1]
    df['latitude'] = df.place.apply(place_to_coordinate, kind='latitude')
    df['longitude'] = df.place.apply(place_to_coordinate, kind='longitude')

    return df

In [99]:
def preprocess(df):
    return (df.pipe(datetimeify))

In [93]:
def preprocess_df(df):
    cleaned = df.pipe(set_hashtags)
    copy = cleaned.copy()
    return preprocess(copy)


In [97]:
def load_df(input_filename):
    raw_df = pd.read_json(input_filename)
    return preprocess(raw_df)

print 'OK'


OK


In [100]:
df = load_df('data-docker/peda.json')
df.head()

# df['created_at'].head()

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,filter_level,geo,id,...,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user
0,NaN,NaN,2016-04-18 02:46:09,"{u'user_mentions': [{u'id': 252569242, u'indic...","{u'media': [{u'source_user_id': 252569242, u's...",0,False,low,NaN,721892546079825921,...,NaN,NaN,0,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/android"" ...",RT @odialetos: QUINTA É FERIADO #ImpeachmentDa...,2016-04-18 02:46:09.757,False,"{u'follow_request_sent': None, u'profile_use_b..."
1,NaN,NaN,2016-04-18 02:46:09,"{u'user_mentions': [{u'id': 437824506, u'indic...","{u'media': [{u'source_user_id': 437824506, u's...",0,False,low,NaN,721892546155298817,...,NaN,NaN,0,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/android"" ...",RT @ferrisss: RT: URGENTE\nPROMESSA CUMPRIDA!\...,2016-04-18 02:46:09.775,False,"{u'follow_request_sent': None, u'profile_use_b..."
2,NaN,NaN,2016-04-18 02:46:09,"{u'user_mentions': [{u'id': 1118076962, u'indi...","{u'media': [{u'source_user_id': 1118076962, u'...",0,False,low,NaN,721892546683781120,...,NaN,NaN,0,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/android"" ...","RT @humdaora: ""pela minha mãe""\n""pela minha es...",2016-04-18 02:46:09.901,False,"{u'follow_request_sent': None, u'profile_use_b..."
3,NaN,NaN,2016-04-18 02:46:09,"{u'user_mentions': [{u'id': 1708160066, u'indi...","{u'media': [{u'source_user_id': 1708160066, u'...",0,False,low,NaN,721892546780229633,...,NaN,NaN,0,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @jsu90ps: Apoyo al gobierno legítimo de @di...,2016-04-18 02:46:09.924,False,"{u'follow_request_sent': None, u'profile_use_b..."
4,NaN,NaN,2016-04-18 02:46:10,"{u'user_mentions': [{u'id': 2533764198, u'indi...",NaN,0,False,low,NaN,721892547312926720,...,NaN,NaN,0,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @SPlKESPINOSA: o estado islâmico nem precis...,2016-04-18 02:46:10.051,False,"{u'follow_request_sent': None, u'profile_use_b..."


In [116]:
tweets = pd.read_json('data-docker/peda.json')
tweets['created_at'] = pd.to_datetime(pd.Series(tweets['created_at']))

tweets.set_index('created_at', drop=False, inplace=True)

tweets.info()
tweets.index = tweets.index.tz_localize('GMT')
tweets.index = tweets.index - DateOffset(hours = 3)
tweets.index

tweets.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 358293 entries, 2016-04-17 15:15:21 to 2016-04-18 02:46:33
Data columns (total 31 columns):
contributors                 0 non-null float64
coordinates                  470 non-null object
created_at                   358293 non-null datetime64[ns]
entities                     358293 non-null object
extended_entities            164504 non-null object
favorite_count               358293 non-null int64
favorited                    358293 non-null bool
filter_level                 358293 non-null object
geo                          470 non-null object
id                           358293 non-null int64
id_str                       358293 non-null int64
in_reply_to_screen_name      4820 non-null object
in_reply_to_status_id        3303 non-null float64
in_reply_to_status_id_str    3303 non-null float64
in_reply_to_user_id          4820 non-null float64
in_reply_to_user_id_str      4820 non-null float64
is_quote_status              358293 

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,filter_level,geo,id,...,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user
created_at,,,,,,,,,,,,,,,,,,,,,
2016-04-17 12:15:21+00:00,NaN,None,2016-04-17 15:15:21,"{u'user_mentions': [{u'id': 2294780467, u'indi...","{u'media': [{u'source_user_id': 2294780467, u'...",0,False,low,None,721718699565178880,...,NaN,NaN,0,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...","RT @GringaBrazilien: Bom Dia Brasil, #Impeachm...",2016-04-17 15:15:21.518,False,"{u'follow_request_sent': None, u'profile_use_b..."
2016-04-17 12:15:21+00:00,NaN,None,2016-04-17 15:15:21,"{u'user_mentions': [{u'id': 87843887, u'indice...",NaN,0,False,low,None,721718699288342528,...,NaN,NaN,0,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/android"" ...",RT @marisascruz: NOSSO HINO!\nNOSSA BANDEIRA!\...,2016-04-17 15:15:21.452,False,"{u'follow_request_sent': None, u'profile_use_b..."
2016-04-17 12:15:21+00:00,NaN,None,2016-04-17 15:15:21,"{u'user_mentions': [{u'id': 206181302, u'indic...",NaN,0,False,low,None,721718699418202112,...,NaN,NaN,0,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/android"" ...","RT @GarotaCiume: Não sou petista, só não sou c...",2016-04-17 15:15:21.483,False,"{u'follow_request_sent': None, u'profile_use_b..."
2016-04-17 12:15:21+00:00,NaN,None,2016-04-17 15:15:21,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,0,False,low,None,721718700496273408,...,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com/#!/download/ipad"" ...",#ThauQuerida #ImpeachmentDay,2016-04-17 15:15:21.740,False,"{u'follow_request_sent': None, u'profile_use_b..."
2016-04-17 12:15:22+00:00,NaN,None,2016-04-17 15:15:22,"{u'user_mentions': [{u'id': 409967714, u'indic...",NaN,0,False,low,None,721718702153023488,...,NaN,NaN,0,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/android"" ...",RT @luadacamz: Eu quero Impeachment desse calo...,2016-04-17 15:15:22.135,False,"{u'follow_request_sent': None, u'profile_use_b..."


In [46]:
tweets30s = tweets['created_at'].resample('1s', how='count')
tweets30s.head()

created_at
2016-04-17 23:46:09+00:00     5
2016-04-17 23:46:10+00:00    17
2016-04-17 23:46:11+00:00    23
2016-04-17 23:46:12+00:00    16
2016-04-17 23:46:13+00:00    21
Freq: S, Name: created_at, dtype: int64

In [47]:
avg = tweets30s.mean()

vincent.core.initialize_notebook()
area = vincent.Area(tweets30s)
area.colors(brew='Spectral')
area.display()

In [49]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=True):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
 
tweet = "RT @medeirosthiiago: testando exemplo TCC! :D http://example.com #ImpeachmentDay"
print(preprocess(tweet))
# ['RT', '@marcobonzanini', ':', 'just', 'an', 'example', '!', ':D', 'http://example.com', '#NLP']


['rt', '@medeirosthiiago', ':', 'testando', 'exemplo', 'tcc', '!', ':D', 'http://example.com', '#impeachmentday']
